In [9]:
from openai import OpenAI 

In [10]:
client = OpenAI(
api_key="sk-b6de1e42df724c84b39a309c5788569d",
base_url="https://api.deepseek.com"
)
DEEPSEEK_MODEL = "deepseek-chat"
print("hello")

hello


In [11]:
# 一次问答
def get_completion(prompt, model=DEEPSEEK_MODEL, temperature=0):
    messages = [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature
    )
    return response.choices[0].message.content

# 多轮对话
def get_completion_from_messages(messages, model=DEEPSEEK_MODEL, temperature=0, max_tokens=None):
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature,
        max_tokens=max_tokens
    )
    return response.choices[0].message.content

In [24]:
import json
# 加载结构化产品信息
product_info_json = """
[
    {
        "brand": "华为",
        "model": "Mate X6",
        "price": 14999,
        "screen_size": "7.93 inches",
        "battery_capacity": "5200 mAh",
        "description": "华为Mate X6是一款拥有7.93英寸可折叠屏幕的旗舰手机，提供极致的视觉体验。"
    },
    {
        "brand": "vivo",
        "model": "X200 Pro",
        "price": 5299,
        "screen_size": "6.78 inches",
        "battery_capacity": "6000 mAh",
        "description": "vivo X200 Pro采用6.78英寸AMOLED屏幕，搭载专业摄影功能，捕捉每一刻精彩。"
    },
    {
        "brand": "小米",
        "model": "Mi 15 Ultra",
        "price": 6499,
        "screen_size": "6.73inches",
        "battery_capacity": "6000 mAh",
        "description": "小米Mi 11 Ultra配备6.73英寸高刷新率屏幕和6000 mAh电池，性能与续航兼具。"
    }
]
"""
product_info_list = json.loads(product_info_json)

In [25]:
def customer_service_assistant(user_message=None, product_info_json=product_info_json, product_info_list=product_info_list):


  #### step1 判断用户问题是否是关于公司的产品
  delimiter = "===="
  system_message = f"""
  您将获得客户服务查询。
  客户服务查询将使用{delimiter}字符作为分隔符。
  确定用户是否正在询问有关特定产品或产品的问题，如果是的话返回Y，否则返回N。
  产品信息如下：
  {product_info_json}
  """
  messages = [
    {'role':'system','content': system_message},
    {'role':'user','content': f"{delimiter}{user_message}{delimiter}"},
  ]
  response2 = get_completion_from_messages(messages)
  print('第一步：判断用户问题是否是关于公司的产品。结果：', response2)
  if response2 == 'N':
    return '抱歉，我们公司产品以外的业务暂时回复不了哦'

  #### step2 识别用户问题中涉及的产品名称
  system_message = f"""
    您将获得客户服务查询。
    客户服务查询将使用{delimiter}字符作为分隔符。
    根据用户的问题，返回用户涉及的产品名称列表。返回的格式只能是json形式，表现为["产品名称1"，"产品名称2"，...],
    其中产品名称只能是'华为、vivo、小米'三个名称中的一个或多个。
    产品信息如下：
    {product_info_json}
  """
  user_message = f"""{user_message}"""
  messages =  [
    {'role':'system','content': system_message},
    {'role':'user','content': f"{delimiter}{user_message}{delimiter}"},
  ]
  response3 = get_completion_from_messages(messages)
  print('第二步：识别用户问题中涉及的产品名称。结果：', response3)

  #### step3 根据名称查询产品信息
  response3_clean = response3.strip().replace('```json', '').replace('```', '').strip()
  for product_name in json.loads(response3_clean):
    for product_dict in product_info_list:
      if product_dict['brand'] == product_name:
        product_rel_json = json.dumps(product_dict, ensure_ascii=False)
        print('第三步：根据名称查询产品信息。结果：', product_rel_json)

  #### step4 根据查询的产品信息回答用户问题
  system_message = f"""
    您是一家销售手机的电商售前助理。
    请以友善平和的语气回答客户的问题。
    客户的问题将使用{delimiter}字符作为分隔符。
  """
  user_message = f"""{user_message}"""
  messages =  [
    {'role':'system','content': system_message},
    {'role':'user','content': f"{delimiter}{user_message}{delimiter}"},
    {'role':'assistant','content': f"""相关产品信息：{product_rel_json}"""}
  ]
  response5 = get_completion_from_messages(messages)
  print('第四步：根据查询的产品信息回答用户问题。结果：', response5)

  #### step5 判断回复给用户的内容是否对用户有用
  user_message = f"""
    用户信息: {delimiter}{user_message}{delimiter}
    代理回复: {delimiter}{response5}{delimiter}
    回复是否足够回答问题
    如果足够，回答 Y
    如果不足够，回答 N
    仅回答上述字母即可
  """
  messages = [
      {'role': 'system', 'content': system_message},
      {'role': 'user', 'content': user_message}
  ]
  response7 = get_completion_from_messages(messages)
  if response7 == 'N':
    print('第五步：判断回复给用户的内容是否对用户有用。结果：对用户无用，需要转人工。')
    return '抱歉，暂时无法回复该问题，将转接人工客服。'
  else:
    print('第五步：判断回复给用户的内容是否对用户有用。结果：对用户有用。')

  return response5
     

In [26]:
input_message = '请问你们有除了小米还有哪些手机吗？能否给我推荐一下？'
output_message = customer_service_assistant(user_message=input_message)

第一步：判断用户问题是否是关于公司的产品。结果： Y
第二步：识别用户问题中涉及的产品名称。结果： ```json
["华为", "vivo"]
```
第三步：根据名称查询产品信息。结果： {"brand": "华为", "model": "Mate X6", "price": 14999, "screen_size": "7.93 inches", "battery_capacity": "5200 mAh", "description": "华为Mate X6是一款拥有7.93英寸可折叠屏幕的旗舰手机，提供极致的视觉体验。"}
第三步：根据名称查询产品信息。结果： {"brand": "vivo", "model": "X200 Pro", "price": 5299, "screen_size": "6.78 inches", "battery_capacity": "6000 mAh", "description": "vivo X200 Pro采用6.78英寸AMOLED屏幕，搭载专业摄影功能，捕捉每一刻精彩。"}
第四步：根据查询的产品信息回答用户问题。结果： 您好！感谢您的咨询。除了小米手机外，我们还销售vivo品牌的手机。比如这款vivo X200 Pro就很不错：

- 6.78英寸AMOLED大屏
- 6000mAh大容量电池
- 专业级摄影功能
- 售价5299元

这款手机特别适合喜欢拍照和追求大屏体验的用户。如果您对这款手机感兴趣，或者想了解其他品牌如OPPO、华为等机型，我很乐意为您详细介绍。您对手机有什么特别的需求吗？比如更注重拍照、游戏性能还是续航？
第五步：判断回复给用户的内容是否对用户有用。结果：对用户有用。


In [27]:
print(output_message)

您好！感谢您的咨询。除了小米手机外，我们还销售vivo品牌的手机。比如这款vivo X200 Pro就很不错：

- 6.78英寸AMOLED大屏
- 6000mAh大容量电池
- 专业级摄影功能
- 售价5299元

这款手机特别适合喜欢拍照和追求大屏体验的用户。如果您对这款手机感兴趣，或者想了解其他品牌如OPPO、华为等机型，我很乐意为您详细介绍。您对手机有什么特别的需求吗？比如更注重拍照、游戏性能还是续航？
